In [ ]:
import os
import pandas as pd
import numpy as np
import random
import nest_asyncio

from vespa.package import ApplicationPackage, Component, Parameter
from vespa.package import Field, FieldSet, RankProfile
from vespa.application import Vespa

nest_asyncio.apply()

VESPA_CONFIG_DIR = "./vespa-app"
VESPA_CONFIG_ZIP = os.path.join(VESPA_CONFIG_DIR, "app_package.zip")

# Comment configurer et feeder votre vespa de projet

doc du client Python Vespa: https://pyvespa.readthedocs.io/en/latest/

Renseignez ici le nom de votre groupe (peu import la casse, je normalise la string pour la suite):

In [ ]:
GROUP_NAME = None

In [ ]:
assert GROUP_NAME, "RENTREZ VOTRE NOM DE GROUPE !"

### Definition du Vespa doctype de votre groupe

Ici vous décrivez les attributs des documents que vous souhaitez indexer. Je propose d'utiliser l'API Pytho de définition de doctype fourni par les gars de Vespa. Sources: https://pyvespa.readthedocs.io/en/latest/application-packages.html#Schema

dans `fields_spec`: mettez la liste des attributs de vos docs avec au minimum un `name` et un `type` pour des champs indicatifs ; ajoutez un `indexing` pour les champs à indexer. Exemple ci-dessous:

## En cas de MàJ de votre doctype
=> il suffit de modifier `fields_spec` et de re-exec le bloc suivant. Vespa se reconfigure à chaud. Seul obstacle: certaines modifs vont déclencher une alerte chez Vespa qui vous demandera d'éditer un fichier `validation-overrides.xml` (juste pour prouver que vous savez ce que vous faites) => ce fichier est généré par le bloc de code ci-dessous => modifier la partie `<allow until ... >` en fonction du message d'erreur que vous rendra le bloc du dessous

In [ ]:
fields_spec = [
    {"name": "id", "type": "string"},
    {"name": "name", "type": "string", "indexing": ["index", "summary", "attribute"]},
    {
        "name": "description",
        "type": "string",
        "indexing": ["index", "summary", "attribute"],
        "index": "enable-bm25",
    },
    {"name": "width", "type": "float", "indexing": ["attribute"]},
]

### Pas toucher ce bloc ...
Executez le => il s'agit de packager les confs de votre appli et de les pousser à votre Vespa

In [ ]:
import re
def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1-\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1-\2', name).lower()

app_name = GROUP_NAME.lower()
vespa_host = "vespa-" + camel_to_snake(GROUP_NAME)
app_package = ApplicationPackage(
    name=app_name, 
    create_query_profile_by_default=False, 
)

app_package.schema.add_fields(*[Field(**field_spec) for field_spec in fields_spec])

app_package.schema.add_field_set(
    FieldSet(name="default", fields=["name", "description"])
)

app_package.schema.add_rank_profile(
    RankProfile(name="default-ranker", first_phase="bm25(description)")
)

app_package.schema.add_rank_profile(
    RankProfile(name="other-ranking", first_phase="bm25(name) + bm25(description)")
)

validation_overrides_str = """
<validation-overrides>
    <allow until='2025-01-21'>indexing-change</allow>
    <allow until='2025-02-21'>content-cluster-removal</allow>
</validation-overrides>

"""
with open(os.path.join(VESPA_CONFIG_DIR, "validation-overrides.xml"), "w") as f:
    f.write(validation_overrides_str)

app_package.to_files(VESPA_CONFIG_DIR)
app_package.to_zipfile(VESPA_CONFIG_ZIP)

!curl -X POST --header "Content-Type: application/zip" \
     --data-binary @$VESPA_CONFIG_ZIP \
     http://$vespa_host:19071/application/v2/tenant/default/prepareandactivate

Vous devriez avoir un message qui dit `Session X for tenant 'default' prepared and activated`. En cas d'erreur quelque part, le message d'erreur devrait vous aiguiller ... mais a priori si vous ne sortez pas des sentiers battus et que vous utilisez le client PyVespa, tout devrait bien se passer

# Vespa est prêt ? Feedons le avec vos data
Le plus simple (mais pas le plus rapide) est de feeder votre Vespa fraîchement configuré avec PyVespa. Pour cela, il suffit de donner la liste des documents à indexer sous la forme d'une liste de `dict` où chaque `dict` contient :
- `id`: l'ID unique de votre doc
- `fields`: un `dict` qui associe vos `fields` avec leur `value`

Exemple avec une liste qui ne contient qu'un doc représenté par les 4 attributs définis dans `fields_spec`:

In [ ]:
data_points = [
    {
        "id": "point1",
        "fields": {
            "id": "cst1",
            "name": "pi",
            "description": "some awesome number",
            "width": 3.14
        }
    }
]

Une fois que vous avez défini vos docs (à vous de faire une fonction pour les lire/process), voici comment les pousser à votre Vespa:

In [ ]:
client = Vespa(url=f"http://{vespa_host}", port=8080)
client.wait_for_application_up(5)
client.feed_async_iterable(data_points, schema=app_name)

## Final check
On vérifie que tout est OK en faisant une requête à Vespa

In [ ]:
resp = client.query(
    {
        "yql": f"select * from {app_name} where userQuery()",
        "hits": 10,
        "query": "pi",
    }
)
resp.json["root"]["children"][0]["fields"]

# Vous êtes prêts pour la suite du projet